In [ ]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import random
from collections import Counter
import numpy as np
# !pip3 install sklearn
from sklearn.linear_model import LinearRegression
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout

In [3]:
xcelLocation = "../xcels/"
# !export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"
RECORD_THRESHOLD = 3000
TESTCASE_NUMBER = 20
alreadyPrep = False

In [4]:
!ls $xcelLocation | grep ".csv" > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [5]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0], parse_dates=["date"])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["symbol", "min", "max"]]
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.at[row, "min"]
        maxim = diffTable.at[row, "max"]
        if(minim != 0):
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diff"] = difs
    df.loc[:,"diff"] = difs
    return df, diffTable

In [6]:
def prepare():
    global alreadyPrep
    global df
    global testSymbol
    
    if alreadyPrep:
        return
    alreadyPrep = True
    
    df0 = readCsv(xcelLocation=xcelLocation, csvFileName="master0")
    df1 = readCsv(xcelLocation=xcelLocation, csvFileName="master1")
    df = pd.concat([df0, df1], ignore_index=True)
    df,_ = addDiff(df=df)
    df = df.set_index('date')
    
    allSymbols = df.symbol.tolist()
    symbols = list(set(df.symbol))[1:]
    print("all symbols: ", len(symbols))
    counts = Counter(allSymbols)
    testSymbols = []
    tmpSymbols = []
    for symbol in symbols:
        if counts[symbol] > RECORD_THRESHOLD:
            tmpSymbols.append(symbol)
    for i in range(TESTCASE_NUMBER):
        ran = random.randint(0, len(tmpSymbols)-1)
        testSymbols.append(tmpSymbols[ran])
        tmpSymbols.remove(tmpSymbols[ran])
    print("test symbol", len(testSymbols))
    

In [7]:
def runSimulator(testDF, minMaxPredictFunction):
    ans = testDF.iloc[len(testDF) - 1]
    testDF = testDF.drop(len(testDF) - 1)
    predictMin, predictMax = minMaxPredictFunction(testDF)

    result = 0
    if predictMin >= ans["min"] and predictMin <= ans["max"]:
        if predictMax <= ans["max"] and predictMax >= ans["min"]:
            result += 1
        else:
            result += 2
    return result

In [8]:
def calcRate(minMaxPredictFunction):
    prepare()
    SS = 0
    SF = 0
    FF = 0
    
    for symbol in testSymbols:
        testDF = df.loc[df["symbol"] == symbol].reset_index(drop=True)
        for i in range(200):
            result = runSimulator(testDF, minMaxPredictFunction)
            if result % 2 == 1:
                SS += 1
            if (result/2) % 2 == 1:
                SF += 1
            if result == 0:
                FF += 1
            testDF = testDF.drop([len(testDF) - 1, len(testDF) - 2, len(testDF) - 3, len(testDF) - 4])
    print("ss, sf, ff: ", SS, " ", SF, " ", FF)
    print("prec ", SS/(SS + SF))
    print("rec ", (SS + SF)/(SS + FF + SF))
            

In [9]:
def rator1(inputDF):
    size = len(inputDF)
    return inputDF.iloc[size - 1]["min"]*102/100 , inputDF.iloc[size - 1]["max"]

In [10]:
def rator2(inputDF):
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [10]:
# calcRate(rator1)

In [11]:
def alarm1():
    prepare()
    df98 = df.loc[df["year"] == 1398].reset_index(drop=True)
    hit5 = df98.loc[
#                     (df98["diff"] <= 10) & (df98["diff"] > 9.8) & 
                    (df98["ending-percent"] >= 3) & (df98["month"] >= 5) &
                    True ].reset_index(drop=True)
    return hit5

In [12]:
hitdf = alarm1()
hitAllSymbols = hitdf.symbol.tolist()
hitSymbols = list(set(hitdf.symbol))[1:]

counts = Counter(hitAllSymbols)
for symbol in hitSymbols:
    if counts[symbol] > 12:
        print(symbol)
# hitdf

symbol                    object
name                      object
amount                     int64
volume                     int64
value                      int64
lastday                  float64
openning                 float64
last-value               float64
last-change              float64
last-percent             float64
ending-value             float64
ending-change            float64
ending-percent           float64
min                      float64
max                      float64
year                       int64
month                      int64
day                        int64
date              datetime64[ns]
diff                     float64
dtype: object
all symbols:  4442
test symbol 20
اعتلاح
كمنگنز
وثخوز
گكوثر
ختوقا
سكرد
ختراك
ثتران
نمرينو
حتوكا
كفپارس
شگل
حتايد
تاپكيش
وپارس
اعتلا
كحافظ
بالبر
كفرآور
توريل
هجرت
حآسا
كاذر
فپنتا
قاسم
شلعاب
غپآذر
غگرجي
تليسه
پخش
خزر
فروس
غالبر
غدشت
ثنوسا
كترام
بكاب
دكوثر
سرود
دفاراح
پلاسك
كپشير
غمينو
كي بي سي
دروز
غپينو
حپترو
دانا
لكما
آسيا
و

In [29]:
df['2001-5']

,symbol,name,amount,volume,value,lastday,openning,last-value,last-change,last-percent,ending-value,ending-change,ending-percent,min,max,year,month,day,diff
date,,,,,,,,,,,,,,,,,,,
2001-05-05,فپنتا,سپنتا,10,5229,43535156,8250.0,8250.0,8394.0,144.0,1.75,8394.0,144.0,1.75,8250.0,8394.0,1380,2,15,1.745455
2001-05-05,فباهنر,مس‌ شهيدباهنر,10,3745,22524498,6007.0,6007.0,6023.0,16.0,0.27,6023.0,16.0,0.27,6007.0,6023.0,1380,2,15,0.266356
2001-05-05,مداران,داده‌پردازي‌ايران‌,11,21645,99536764,4580.0,4600.0,4598.0,18.0,0.39,4598.0,18.0,0.39,4598.0,4600.0,1380,2,15,0.043497
2001-05-05,غپينو,پارس‌ مينو,11,6879,56804122,8249.0,8248.0,8300.0,51.0,0.62,8300.0,51.0,0.62,8248.0,8300.0,1380,2,15,0.630456
2001-05-05,دكيمي,كيميدارو,11,7519,60551338,8014.0,8016.0,8149.0,135.0,1.68,8149.0,135.0,1.68,8016.0,8149.0,1380,2,15,1.659182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-05-29,دجابر,داروسازي‌ جابرابن‌حيان‌,20,7984,126988718,16000.0,15999.0,15800.0,-200.0,-1.25,15800.0,-200.0,-1.25,15800.0,15999.0,1380,3,8,1.259494
2001-05-29,غپارس,روغن‌ نباتي‌ پارس‌,20,11901,191371625,16080.0,16080.0,16081.0,1.0,0.01,16081.0,1.0,0.01,16079.0,16081.0,1380,3,8,0.012439
2001-05-29,سپاها,سيمان‌سپاهان‌,21,9290,395474800,42570.0,42570.0,42570.0,0.0,0.00,42570.0,0.0,0.00,42569.0,42570.0,1380,3,8,0.002349
